In [1]:
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('../data/data_for_Rubert/my_dataset.csv', lineterminator='\n', skipinitialspace=True)

In [3]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str).str.replace('\n', ' ', regex=False)
    df[col] = df[col].str.strip()

print(df.head())

                                             comment  toxic
0                 Верблюдов-то за что? Дебилы, бл...    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                            Собаке - собачья смерть    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0


In [4]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [5]:
russian_stopwords = stopwords.words('russian')
print(f"Количество стоп-слов: {len(russian_stopwords)}")
print(russian_stopwords[:20]) 

Количество стоп-слов: 151
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты']


In [6]:
def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation and stopwords

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """

    text = text.lower()
    text = re.sub("<.*?>", "", text)
    text = text.replace('-', ' ')
    text = text.replace('  ', ' ')
    text = "".join([c for c in text if c not in string.punctuation])
    splitted_text = [word for word in text.split() if word not in russian_stopwords]
    text = " ".join(splitted_text)
    return text


df["comment_cleaned"] = df["comment"].apply(data_preprocessing)
df.head()

,comment,toxic,comment_cleaned
0,"Верблюдов-то за что? Дебилы, бл...",1.0,верблюдов дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть,1.0,собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страницу обнови дебил это оскорбление доказанн...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,убедил 6 страничный пдф скрипалей отравила рос...


In [7]:
df['toxic'].value_counts()

toxic
0.0    9586
1.0    4826
Name: count, dtype: int64

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny-toxicity")
# model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny-toxicity")

/home/maxim/DS/projects/NLP-proj/.nlp_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
test = tokenizer(df['comment_cleaned'][0])
print(test)
print(df['comment_cleaned'][0])

{'input_ids': [2, 314, 1813, 18814, 920, 9691, 1395, 12330, 700, 313, 869, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
верблюдов дебилы бл


In [14]:
encoded_train_content =  df['comment_cleaned'].apply(
    lambda x: tokenizer(x, max_length=64, truncation=True, padding='max_length')
).values

In [15]:
encoded_train_content

array([{'input_ids': [2, 314, 1813, 18814, 920, 9691, 1395, 12330, 700, 313, 869, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
       {'input_ids': [2, 333, 16255, 2520, 2389, 733, 3914, 14811, 650, 24174, 4536, 2095, 8918, 7880, 14653, 324, 4375, 815, 679, 333, 16255, 5021, 333, 644, 3881, 333, 16255, 5021, 3283, 21665, 1090, 1129, 27549, 2246, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0,